# Example queries for geolocations on COVID-19 Knowledge Graph
[Work in progress]

This notebook demonstrates how to run Cypher queries in a Jupyter Notebook by connecting to a database server.

In [1]:
import datetime
import pandas as pd
from py2neo import Graph

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

### List Node Metadata

In [4]:
query = """
MATCH (n:NodeMetadata)
RETURN n.name, n.shortDescription, n.description, n.example, n.definitionSource, n.dataProviders
"""
graph.run(query).to_data_frame()

,n.name,n.shortDescription,n.description,n.example,n.details
0,Location,Geographic location,A geograpic location,"World, ..., Country, State, Country, City, Cru...",None
1,World,The World,Top level location,None,None
2,UNRegion,Continental regions,Continental regions according to the M49 stan...,Americas,https://unstats.un.org/unsd/methodology/m49/
3,UNSubRegion,Subcontinental regions,Subcontinental regions according to the M49 st...,Latin America and the Caribbean,https://unstats.un.org/unsd/methodology/m49/
4,UNIntermediateRegion,Subdivisions of subcontinental regions,Subdivisions of subcontinental regions accordi...,Caribbean,https://unstats.un.org/unsd/methodology/m49/
5,Country,Countries and dependent Territories,Countries and dependent Territories defined b...,United States,http://www.geonames.org/
6,Admin1,"State, Province, Municipality","First administrative divisions, e.g, State, Pr...",California,http://www.geonames.org/
7,Admin2,County,Second administrative divisions: County in the US,San Diego County,http://www.geonames.org/
8,City,City,City,San Diego,http://www.geonames.org/
9,PostalCode,Postal Code,E.g.,a ZIP Code is a postal code used by the Unite...,http://purl.obolibrary.org/obo/OPMI_0000120


### Run a full text search for location names

In [5]:
query = """
CALL db.index.fulltext.queryNodes("locations", "Los Angeles") YIELD node, score
RETURN node.name, node.population, labels(node), score
"""
df = graph.run(query).to_data_frame()
df.head(25)

,node.name,node.population,labels(node),score
0,Los Angeles,1144.0,"[Location, City]",5.888921
1,Los Angeles,4217.0,"[Location, City]",5.888921
2,Los Angeles,1616.0,"[Location, City]",5.888921
3,Los Angeles,1780.0,"[Location, City]",5.888921
4,Los Angeles,250.0,"[Location, City]",5.888921
5,Los Angeles,342.0,"[Location, City]",5.888921
6,Los Angeles,4054.0,"[Location, City]",5.888921
7,Los Angeles,3971883.0,"[Location, City]",5.888921
8,East Los Angeles,126496.0,"[Location, City]",4.704631
9,Fraccionamiento los Angeles,6282.0,"[Location, City]",4.704631
